# 1) Converting and Merging TIFF Files with Nodata Value Adjustment

    This code converts zero values to a predefined no-data value in a collection of TIFF files, then merges these modified files into a single TIFF. It utilizes the GDAL library for reading and writing geospatial raster data, and numpy for array manipulation. After converting zero values to the specified no-data value (-9999) in each file, it merges the modified files into one using the gdal_merge.py script, indicating a successful operation through print statements.

In [ ]:
from osgeo import gdal
import numpy as np
import os
import subprocess

tif_folder = '/path/to/save/indices/'

tif_files = [file for file in os.listdir(tifv_folder) if file.endswith(".tif")]

converted_files = []
nodata_value = -9999 

for file in tif_files:
    input_path = os.path.join(tif_folder, file)
    output_path = os.path.join(tif_folder, "nan_converted_" + file)

    dataset = gdal.Open(input_path)
    band = dataset.GetRasterBand(1)

    data = band.ReadAsArray()
    data = np.where(data == 0, nodata_value, data)

    driver = gdal.GetDriverByName('GTiff')
    out_dataset = driver.Create(output_path, band.XSize, band.YSize, 1, band.DataType)
    out_band = out_dataset.GetRasterBand(1)

    out_band.SetNoDataValue(np.nan)
    out_band.WriteArray(data)

    out_dataset.SetGeoTransform(dataset.GetGeoTransform())
    out_dataset.SetProjection(dataset.GetProjection())
    out_dataset.FlushCache()

    converted_files.append(output_path)

output_file = "/path/to/save/mosaic/index_name.tif"

gdal_merge_path = r'path/to/gdal_merge.py' 
cmd = f"python {gdal_merge_path} -ps 20 -20 -o {output_file}"

subprocess.call(cmd.split() + converted_files)

print("Merging was successful")